In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [4]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Nice to meet you, and welcome to the chat. Thanks for the introduction—happy to help with whatever you’re curious about.\n\nHere are a few ways I can assist:\n- Explain topics in simple terms\n- Help with writing (emails, essays, stories, resumes)\n- Plan or brainstorm (projects, trips, events)\n- Solve problems (math, coding, logic)\n- Learn or practice a language (translate, practice conversations)\n- Summarize or analyze articles or documents\n- Generate ideas (creative prompts, recipes, workouts)\n\nTell me what you’d like to do, or just say hi again and we can chat about anything you’re interested in. If you want, share a bit about your interests or a goal you have, and I’ll tailor the help.'

In [5]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [7]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [10]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarkyassistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. 

In [11]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://edwarddonner.com")

"# Edward Donner's Website: The LLM Playground of a Code-DJ\n\nWelcome to Ed’s digital lair, where coding meets barely-there DJ skills and a bizarre fondness for Hacker News nodding. Ed's a big brain behind Nebula.io, an AI startup that pretends to help recruiters find human potential (or robots pretending to be humans). Previously sold his AI baby untapt, so yeah, he's legit.\n\n**Cool Stuff Here:**\n- **Connect Four & Outsmart:** Where large language models fight each other like sci-fi diplomats with sinister plans.  \n- Proprietary AI matchmaking that’s patented—even AI needs patents now.  \n- AI courses & executive briefings dated into 2025 — Ed’s apparently time-traveling.  \n\n**Newsflash:**\n- Latest buzz about Gen AI and Agentic AI scaling on AWS (because why not?)  \n- More courses to become an LLM overlord, if that’s your jam.  \n\nWant to connect with a guy who codes, DJs (poorly), and outsmarts AI? Ed’s got your back. Just don’t ask him to drop a beat."

In [13]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

# Edward Donner's Website: The AI Playground of a Code-DJ Hybrid

Edward (Ed) is a self-professed LLM tinkerer who moonlights as an amateur electronic music producer and a barely-practicing DJ. His main gig? Co-founder and CTO of Nebula.io, where AI meets talent discovery—think matchmaking but for job recruiters and candidates, powered by proprietary (patented!) LLM wizardry.

**Cool toys on the site:**
- **Connect Four** — No, not the childhood favorite, but probably a code project.
- **Outsmart** — A clever LLM smackdown arena where AI models battle with diplomacy and deviousness. Basically, AI throwing shade in binary.

**Recent blog-ish announcements:**
- September 2025: Chat about Gen AI and its AWS scaling adventures.
- May 2025: Sneak peek into his courses for budding LLM leaders.
- April 2025: Drop of a full-fledged course on Agentic AI Engineering.

If you want to get in touch or stalk his LLM exploits, he's got all the classic social bait—LinkedIn, Twitter, Facebook—and even a newsletter for the truly committed. Ed’s site is your go-to for a mix of AI brainpower, a sprinkle of DJ dreams, and some serious talent-matching mojo.

In [17]:
display_summary("https://cnn.com")

# CNN: The Never-Ending News Buffet

Welcome to CNN, where breaking news is your new best friend and ads are your unavoidable sidekick. Want to complain about a slow-loading video or an ad that never quite gets the memo? Great, they have a whole form for that. If you’re into politics, weather, sports, or keeping up with every war and conflict on the planet, they pretty much have you covered—then some. 

Basically, it’s a 24/7 news extravaganza packed with everything from Ukraine-Russia and Israel-Hamas war updates to the latest celebrity gossip, fitness tips, and market chaos. If you’re tired of peaceful ignorance, CNN’s got you covered in 15 different categories and languages.

In short: CNN is the buffet of news — sometimes burnt, sometimes fresh, but always loaded with way too many options and ads. Bon appétit!

In [18]:
display_summary("https://anthropic.com")

Anthropic: The AI company that’s all about making Artificial Intelligence less likely to turn into Skynet. They’re pushing the envelope with models like Claude Sonnet 4.5 (apparently the world’s best for agents and coding—buzzword bingo alert) and the new Claude Haiku 4.5 for managing context. Their mission? Keep AI safe and beneficial, while simultaneously reminding us they’re taking "intentional pauses" because, hey, no one wants a robot rebellion. Also, they’re super proud to be a public benefit corporation, because saving humanity sounds better on a resume.